In [ ]:
import csv
import gzip
import time
import os
import pandas as pd
import re
from numpy import where as npwhere
from functools import reduce

def loadFileRemoveMts(replicate,replicaname):
        bxdf = pd.read_csv(
            replicate,
            sep='\t', 
            names=['chr', 'position', 'strand', replicaname+'mets', replicaname+'nomets', 'context', 'boxID', 'side'],
            usecols=['chr', 'position', 'strand', 'boxID',replicaname+'mets', replicaname+'nomets', 'context']
                )
        bxdf.drop(bxdf[(bxdf.chr == 'Mt') | (bxdf.chr == 'Pt')].index, inplace=True)
        
        return bxdf
# 

def sumReplicatesValues(bxdf, metsNames,nometsNames,experimentCondition):
    bxdf[experimentCondition+'mets'] = bxdf[metsNames].sum(axis=1)
    bxdf[experimentCondition+'nomets'] = bxdf[nometsNames].sum(axis=1)
    print(metsNames + nometsNames)
    bxdf = df_merged.drop(columns=metsNames + nometsNames)
    return bxdf[bxdf[experimentCondition+'mets'] + bxdf[experimentCondition+'nomets'] >= 4]


In [ ]:
working_folder = '/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps'
intersection_file = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'
basedmrfolder = '/home/joaquin/projects/methylation/data/DMRs'
experimentsClasification = {}
for root, dirs, files in os.walk(working_folder):

    for file in files:
        if 'CX_report.txt.gz' in file:
            replicate, hour, condition =root.strip().split('/')[-3:]
            experimentName = hour+condition
            experimentsClasification.setdefault(experimentName, [])
            targetMetFilename = root+'/'+file
            finalnamedestination = basedmrfolder+'/'+experimentName+replicate+'_CX_report.txt.gz'
            outputfile = targetMetFilename[:-3] + 'peakMets.tsv'
            experimentsClasification[experimentName].append(outputfile)


            

In [ ]:
allthedata = []
for experimentCondition in experimentsClasification:
    listOfDfReplicatesPercentageOfMetCs = []
    metsNames = []
    nometsNames = []
    listOfDfReplicatesCualitative = []
    listOfNamesReplicates = []
    for replicatePath in experimentsClasification[experimentCondition]:

        replicaname =  experimentCondition+replicatePath.split('/')[-4]
        replicateDf = loadFileRemoveMts(replicatePath,replicaname)
        metsNames.append(replicaname+'mets')
        nometsNames.append(replicaname+'nomets')

#         listOfNamesReplicates.append(replicaname)
        listOfDfReplicatesPercentageOfMetCs.append(replicateDf)

        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), listOfDfReplicatesPercentageOfMetCs).fillna(0)
    print(df_merged)
    df_merged = sumReplicatesValues(df_merged, metsNames,nometsNames,experimentCondition)

    
    allthedata.append(df_merged)
finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), allthedata).fillna(0)


In [ ]:
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'
peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])
for context in ['CHH','CHG','CG']:
    df = finaldf[(finaldf['context'] == context)]
    values = df.iloc[:,4:]
    sumdf = values.groupby('boxID', as_index=False).sum()
    peaks = peaks.merge(sumdf, on='boxID', how='outer')
    peaks =peaks.fillna(0)
    for experimentCondition in ['1Mock','6Mock','24Mock','1ACC','6ACC','24ACC','1JA','6JA','24JA']:
        peaks[experimentCondition] = peaks[experimentCondition+'mets'] / (peaks[experimentCondition+'nomets'] + peaks[experimentCondition+'mets'])
        peaks = peaks.drop(columns=[experimentCondition+'mets',experimentCondition+'nomets'])
    peaks.to_csv(f'{context}_BSMet__sum_all_values_and_mean.tsv',sep='\t', index=False)

In [ ]:
finaldf

In [ ]:
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'
peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])

df = finaldf[(finaldf['context'] == 'CG')]
values = df.iloc[:,4:]
sumdf = values.groupby('boxID', as_index=False).sum()
peaks = peaks.merge(sumdf, on='boxID', how='outer')
peaks =peaks.fillna(0)
for experimentCondition in ['1Mock','6Mock','24Mock','1ACC','6ACC','24ACC','1JA','6JA','24JA']:
    peaks[experimentCondition] = peaks[experimentCondition+'mets'] / (peaks[experimentCondition+'nomets'] + peaks[experimentCondition+'mets'])
    peaks = peaks.drop(columns=[experimentCondition+'mets',experimentCondition+'nomets'])
# peaks.to_csv(f'non-CG_BSMet__sum_all_values_and_mean.tsv',sep='\t', index=False)

In [ ]:
peaks.iloc[:,4:]

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt
# import seaborn
import seaborn as sns
%matplotlib inline
plotmatr = peaks.iloc[:,4:]
plotmatr = plotmatr[(plotmatr >= 0.1).all(1)]
plotmatr = plotmatr[(plotmatr <= 0.9).all(1)]
bplot = sns.boxplot(data=plotmatr, 
                 width=0.5,
                 palette="colorblind")

In [ ]:
plot_file_name="boxplot_and_swarmplot_with_seaborn.jpg"
 
# save as jpeg
bplot.figure.savefig('boxplot_Sincolas01_cg_context.jpg',
                    format='jpeg',
                    dpi=300)